<a href="https://colab.research.google.com/github/prubinstreit/Cryo_Scraper/blob/main/Fairfax_Cryo_Bank_Scraper_Each_Profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
# Running this notebook on Google Colab? Then run this cell! Otherwise, please use the cell above!

!pip install selenium
!pip install helium
!apt-get update 
!apt install chromium-chromedriver
!pip install requests


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [278]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from datetime import date
import re
import time

import pandas as pd
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from helium import *
import requests

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


In [279]:
url = 'https://fairfaxcryobank.com/search/searchresults.aspx'
driver.get(url)
element = driver.find_element(By.CLASS_NAME, "submit")
element.click()

In [280]:
time.sleep(1)
results_html = BeautifulSoup(driver.page_source, 'html.parser')
h = results_html.find_all('div', attrs={'class': 'total'})[0]
pages = h.get_text()[-2:]
pages = int(pages)
pages

36

In [281]:
def next_page():
  try:
    #driver.switch_to().accept()
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='content']/div/div[3]/div/div[1]/div[4]/div[2]/a"))).click()
    #driver.implicitly_wait(10)
    #driver.switch_to().alarm().accept()
  except:
    pass

In [282]:
def profiles():
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  el = soup.find_all(class_='left', href=True)
  blocks = len(el)
  url_profile = []
  for i in range(blocks):
    url_profile.append(el[i]['href'])
  df1=pd.DataFrame({'ID':url_profile, 'site':"https://fairfaxcryobank.com/search/"})

  list = [soup, df1]
  del list
  df1['url']=df1.site.str.cat(df1.ID)
  return df1

#profiles()

In [283]:
df_final = profiles()
df_final = pd.DataFrame(df_final['url'])
df_final
df_list

,url
0,https://fairfaxcryobank.com/search/donorprofil...
2,https://fairfaxcryobank.com/search/donorprofil...
3,https://fairfaxcryobank.com/search/donorprofil...
4,https://fairfaxcryobank.com/search/donorprofil...
5,https://fairfaxcryobank.com/search/donorprofil...
...,...
5,https://fairfaxcryobank.com/search/donorprofil...
6,https://fairfaxcryobank.com/search/donorprofil...
7,https://fairfaxcryobank.com/search/donorprofil...
8,https://fairfaxcryobank.com/search/donorprofil...


In [284]:
#get all profile url's from search page
df_list=[]
for i in range(1):
  df_final = profiles()
  df_final = pd.DataFrame(df_final['url'])
  df_list = df_final
  del df_final

for j in range(1,pages):
  next_page()
  time.sleep(1)
  df_final = profiles()
  df_final = pd.DataFrame(df_final['url'])
  df_list = pd.concat([df_list,df_final])
  del df_final 
#now = datetime.now()
#df_list['Date']= now
df_list
user_num = len(df_list)

In [285]:
def page_data(str):
  driver.get(str)
  soup = BeautifulSoup(driver.page_source,'html.parser')
  #descriptives
  data1 = soup.find('ul', attrs={'class': 'list-info'})
  i = 0
  new = [x.get_text() for x in data1.find_all('li')]
  data1 = {"Ancestry": new[0][9:],
          "Ethnic Background": new[1][18:],
          "Degree":new[2][7:],
          "Occupation":new[3][11:],
          "Interests":new[4][10:],
          "Photo Matching":new[6][15:],
          "CMV Status":new[7][11:],
          "Age Range at Donation":new[8][22:],
          "Astrological Sign":new[9][18:],
          "Favorite Subject":new[10][17:],
          "Religion":new[11][9:],
          "Favorite Pet":new[12][13:],
          "Personal Goals":new[13][15:],
          "Talents/Hobbies":new[14][16:],                          
          }
  df_extra = pd.DataFrame(data1,index=[0])
  #print(df_extra)

  #Vials and Price Availabilty
  bl01 = [x.get_text() for x in soup.find_all('tr', attrs={'class': 'sys-product-cnt'})]
  arr = [a.split("\n") for a in bl01]

  if len(arr)==0:
   data = {"IUI": '',
          "IUI ART": '',
          "ICI":'',
          "ICI ART":'',
          "IVF":''
         }
  if len(arr)==1:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": '',
          "ICI":'',
          "ICI ART":'',
          "IVF":''
         }
  elif len(arr)==2:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":'',
          "ICI ART":'',
          "IVF":''
         }
  elif len(arr)==3:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":''.join(arr[2]),
          "ICI ART":'',
          "IVF":''
        }
  elif len(arr)==4:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":''.join(arr[2]),
          "ICI ART":''.join(arr[3]),
          "IVF":''
        }
  elif len(arr)==5:
   data = {"IUI": ''.join(arr[0]),
          "IUI ART": ''.join(arr[1]),
          "ICI":''.join(arr[2]),
          "ICI ART":''.join(arr[3]),
          "IVF":''.join(arr[4])
        }
  df = pd.DataFrame(data,columns = ['IUI', 'IUI ART','ICI','ICI ART','IVF'], index=[0])

  #height weight and hair
  bl01 = [x.get_text() for x in soup.find_all('div', attrs={'class': 'info'})]
  arr = [a.split("\n") for a in bl01]
  hair = arr[4][2]+ arr[4][1]
  hair = hair.split('\t')
  hair = hair[16]+hair[8]
  #print(hair)
  data = {'Height':  arr[0],
          'Weight':  arr[1],
          'Eye Color':  arr[2],
          'Skin Tone':  arr[3],
          'Hair':  hair,
          'Blood Type':  arr[5],
      }
  #print(data)
  df1 = pd.DataFrame(data)
  df_final= pd.merge(df,df1,left_index=True, right_index=True)
  df_final= pd.merge(df_final,df_extra,left_index=True, right_index=True)
  df_final['ID']=str[-15:]

  return df_final

In [286]:

page_data(df_list.iloc[250,0])


,IUI,IUI ART,ICI,ICI ART,IVF,Height,Weight,Eye Color,Skin Tone,Hair,Blood Type,Ancestry,Ethnic Background,Degree,Occupation,Interests,Photo Matching,CMV Status,Age Range at Donation,Astrological Sign,Favorite Subject,Religion,Favorite Pet,Personal Goals,Talents/Hobbies,ID
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'11""(180 cm)",166 lb(75 kg),Brown,Light,Brown/Wavy,O+,Caucasian,Croatian/German-Norwegian-Swedish,Pursing BEE/Bach Elect Eng,Student/Transporation Services,"Camping, Cars, Hiking, Hockey, Rock Climbing, ...",Yes,Negative,18-22,Leo,"Natural Science, Technology",Christian,Cat,"Financial Security, To Help People","Cars, Hockey, Soccer",number=6042&s=1


In [294]:
#aggregate each user data (will take a long time)
df_list_final=[]

df_final = page_data(df_list.iloc[0,0])
df_list_final = df_final
#del df_final

#print(df_list_final)
df_list_final = df_list_final.drop(range(1))
for j in range(user_num):
  #driver.get(df_list.iloc[j,0])
  #time.sleep(1)
  print("j is:",j,"url is",df_list.iloc[j,0])
  df_final = page_data(df_list.iloc[j,0])
  df_list_final = pd.concat([df_list_final,df_final])

  print(df_list_final)
  del df_final
 
now = datetime.now()
df_list_final['Date']= now
#df_list_final['ID'] = df_list['url']
df_list_final

#print(df_list.head(10))
#print("df_list size:",len(df_list))

Streaming output truncated to the last 5000 lines.
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=5346&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=6305&s=1
0  IUI ARTIntrauterine Insemination Ready individ...  ...  number=6289&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=4300&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=6189&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=4287&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=5145&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=4814&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=5833&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=6033&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=5067&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1...  ...  number=6008&s=1
0  IUIIntrauterine Insemination Ready / Washed/ 1

,IUI,IUI ART,ICI,ICI ART,IVF,Height,Weight,Eye Color,Skin Tone,Hair,Blood Type,Ancestry,Ethnic Background,Degree,Occupation,Interests,Photo Matching,CMV Status,Age Range at Donation,Astrological Sign,Favorite Subject,Religion,Favorite Pet,Personal Goals,Talents/Hobbies,ID,Date
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'8""(172 cm)",172 lb(78 kg),Brown,Medium,Black/Straight,A+,Asian,Nepalese/Nepalese,PhD/Analytical Chemistry,Postdoctoral fellow,"Family Activities, Indiviaual Sports, Team Sports",Yes,Positive,29-34,Scorpio,Natural Science,Hindu,Dog,"Further Education, Rewarding Career, To Be Hap...","Badminton, Cricket, Tennis, Volleyball",number=6281&s=1,2022-02-24 19:49:55.563353
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",213 lb(96 kg),Brown,Medium Light,Black/Straight,O+,Multi,Mexican/English-Mexican,MS/Engineering Management,Engineer,"Baseball, Cooking, Guitar, Piano, Singing",Yes,Positive,29-34,Taurus,Mathematics,Christian,Dog,"Further Education, Rewarding Career, Success","Baseball, Cooking, Piano, Singing musicals, So...",number=6087&s=1,2022-02-24 19:49:55.563353
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'9""(175 cm)",171 lb(77 kg),Brown,Light,Black/Straight,O+,Asian,Vietnamese/Vietnamese,PhD/Material Sci & Engineering,Postdoctoral researcher,"Comics, Individual Sports, Photography, Readin...",Yes,Positive,29-34,Scorpio,"History, Languages, Mathematics, Natural Scien...",Other (Atheist),Other (Turtle),"Financial Security, Further Education, Improve...","Adventure novels, Badminton, Japanase manga, L...",number=5866&s=1,2022-02-24 19:49:55.563353
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'10""(177 cm)",163 lb(73 kg),Brown,Light,Black/Straight,B+,Asian,Chinese/Chinese,MS/Math and Computational Fin,Data Scientist/Programmer Anal,"Basketball, Running, Tennis",Yes,Positive,35-39,Virgo,"Mathematics, Technology",Christian,Dog,"Further Education, Success, To Help People","Basketball, Tennis",number=5852&s=1,2022-02-24 19:49:55.563353
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",266 lb(120 kg),Brown,Medium,Black/Wavy,O+,Latino,Mexican/Mexican,BA/Psyc & Spanish,Student,"Farming, Gardening, Helping environment, Outdo...",Yes,Positive,18-22,Virgo,"Languages, Social Science",Christian (Catholic),Other (Dogs and Cats),Financial Security,"Experiencing new cultures, Riding motocycle, W...",number=5834&s=1,2022-02-24 19:49:55.563353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,IUIIntrauterine Insemination Ready / Washed/ 1...,IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",186 lb(84 kg),Blue,Medium Light,Blond/Straight,O+,Caucasian,English/Norwegian,BA/History & Economics,Student,"Camping, Fitness, Music",Yes,Positive,18-22,Aries,"History, So

In [306]:
df_list_final.replace(regex=['number='], value='', inplace=True)
df_list_final.replace(regex=['&s=1'], value='',inplace = True)
df_list_final.replace(regex=['er='], value='',inplace = True)
df_list_final.replace(regex=['Intrauterine Insemination Ready / Washed/'], value='', inplace = True)
df_list_final.replace(regex=['Intrauterine Insemination Ready / Washed/'], value='', inplace = True)
df_list_final.replace(regex=['Intrauterine Insemination Ready / Washed/'], value='', inplace = True)

df_list_final.replace(regex=['Add to Cart'], value='', inplace = True)
df_list_final.replace(regex=['12345678910'], value=' Max10 ', inplace = True)
df_list_final.replace(regex=['246810'], value=' EvenMax10 ', inplace = True)
df_list_final


,IUI,IUI ART,ICI,ICI ART,IVF,Height,Weight,Eye Color,Skin Tone,Hair,Blood Type,Ancestry,Ethnic Background,Degree,Occupation,Interests,Photo Matching,CMV Status,Age Range at Donation,Astrological Sign,Favorite Subject,Religion,Favorite Pet,Personal Goals,Talents/Hobbies,ID,Date
0,"IUI 10 million total motile cells$1,220Max10 1...",IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'8""(172 cm)",172 lb(78 kg),Brown,Medium,Black/Straight,A+,Asian,Nepalese/Nepalese,PhD/Analytical Chemistry,Postdoctoral fellow,"Family Activities, Indiviaual Sports, Team Sports",Yes,Positive,29-34,Scorpio,Natural Science,Hindu,Dog,"Further Education, Rewarding Career, To Be Hap...","Badminton, Cricket, Tennis, Volleyball",6281,2022-02-24 19:49:55.563353
0,"IUI 10 million total motile cells$1,095Max10 1...",IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",213 lb(96 kg),Brown,Medium Light,Black/Straight,O+,Multi,Mexican/English-Mexican,MS/Engineering Management,Engineer,"Baseball, Cooking, Guitar, Piano, Singing",Yes,Positive,29-34,Taurus,Mathematics,Christian,Dog,"Further Education, Rewarding Career, Success","Baseball, Cooking, Piano, Singing musicals, So...",6087,2022-02-24 19:49:55.563353
0,"IUI 10 million total motile cells$1,220Max10 1...",IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'9""(175 cm)",171 lb(77 kg),Brown,Light,Black/Straight,O+,Asian,Vietnamese/Vietnamese,PhD/Material Sci & Engineering,Postdoctoral researcher,"Comics, Individual Sports, Photography, Readin...",Yes,Positive,29-34,Scorpio,"History, Languages, Mathematics, Natural Scien...",Other (Atheist),Other (Turtle),"Financial Security, Further Education, Improve...","Adventure novels, Badminton, Japanase manga, L...",5866,2022-02-24 19:49:55.563353
0,"IUI 10 million total motile cells$1,095Max10 1...",IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"5'10""(177 cm)",163 lb(73 kg),Brown,Light,Black/Straight,B+,Asian,Chinese/Chinese,MS/Math and Computational Fin,Data Scientist/Programmer Anal,"Basketball, Running, Tennis",Yes,Positive,35-39,Virgo,"Mathematics, Technology",Christian,Dog,"Further Education, Success, To Help People","Basketball, Tennis",5852,2022-02-24 19:49:55.563353
0,"IUI 10 million total motile cells$1,2201234567...",IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",266 lb(120 kg),Brown,Medium,Black/Wavy,O+,Latino,Mexican/Mexican,BA/Psyc & Spanish,Student,"Farming, Gardening, Helping environment, Outdo...",Yes,Positive,18-22,Virgo,"Languages, Social Science",Christian (Catholic),Other (Dogs and Cats),Financial Security,"Experiencing new cultures, Riding motocycle, W...",5834,2022-02-24 19:49:55.563353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"IUI 10 million total motile cells$1,220Notify Me",IUI ARTIntrauterine Insemination Ready individ...,ICIIntracervical Insemination Ready / Unwashed...,ICI ARTIntracervical Insemination Ready indivi...,IVFIn Vitro Fertilization Unit / Unwashed/ 5 m...,"6'0""(182 cm)",186 lb(84 kg),Blue,Medium Light,Blond/Straight,O+,Caucasian,English/Norwegian,BA/History & Economics,Student,"Camping, Fitness, Music",Yes,Positive,18-22,Aries,"History, Social Science",Christian (Episcopalian),Cat,"

In [295]:
today = datetime.now()
date1 = today.strftime("%b-%d")
from google.colab import files
df_list_final.to_csv('%s.csv'%date1) 
files.download('%s.csv'%date1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>